In [3]:
antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"

In [1]:
from Tool_Functions.cleaning_data import * 

from Clustering.new_data_set_all import * 
from Clustering.viualize_datas import * 
from Clustering.new_data_set_all import *

from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = file_to_dataframe(antoine + "df_all_clusters.csv")

In [11]:
df

,ID_ABONNE,DATE_ACTE_REEL,DATE_DEMARRAGE_PROMO,TYPE_PROMON,STATUT_FIN_M_MOINS_1,DATE_PRISE_EFFET,DATE_FIN_ABO_PREC,DATE_FIN_REABO,REGION,FORMULE_PREC,FORMULE_REABO,CANAL_DISTRIB,ENSEIGNE,MOYEN_PAIEMENT,REABO_AVANT_ECHEANCE,DELAI_REABO,NB_APPARITIONS,MOY_DELAI,Cluster_8
0,51276418001,2022-06-19,2022-06-19,ODD 30 jours TC,1.0,2022-06-19,2022-05-31,2022-09-18,Sénégal - Divers,AFRIQUE - EVASION & CANAL+,AFRIQUE - EVASION & CANAL+,Réseau non physique,Mobile Payment,Cash,0,19,9,17.111111,4
1,51276418001,2021-12-01,2021-12-01,ODD 21 jours TC,0.0,2021-12-01,2021-08-23,2022-02-28,Sénégal - Divers,AFRIQUE - EVASION,AFRIQUE - EVASION & CANAL+,Réseau non physique,Mobile Payment,Cash,0,100,9,17.111111,4
2,51276418001,2021-05-15,2021-05-15,ODD 15 jours TC,1.0,2021-05-15,2021-04-30,2021-06-14,Dakar,AFRIQUE - EVASION & CANAL+,AFRIQUE - EVASION & CANAL+,Réseau non physique,Mobile Payment,Cash,0,15,9,17.111111,4
3,51276418001,2022-01-16,2022-01-16,ODD 15 jours TC,1.0,2022-03-01,2022-02-28,2022-05-31,Sénégal - Divers,AFRIQUE - EVASION & CANAL+,AFRIQUE - EVASION & CANAL+,Réseau non physique,Mobile Payment,Cash,1,-43,9,17.111111,4
4,51276418001,2021-07-24,2021-07-24,ODD 15 jours TC,0.0,2021-07-24,2021-06-14,2021-08-23,Dakar,AFRIQUE - EVASION & CANAL+,AFRIQUE - EVASION,Réseau non physique,Mobile Payment,Cash,0,40,9,17.111111,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982347,51458332904,2021-02-09,2021-02-09,PAS_ODD,1.0,2021-02-09,2021-02-07,2022-02-08,Dakar,AFRIQUE - EVASION & CANAL+,AFRIQUE - EVASION & CANAL+,Réseau en propre,Boutiques CANAL+,Cash,0,2,3,3.666667,4
982348,51458332904,2023-02-16,2023-02-16,PAS_ODD,1.0,2023-02-16,2023-02-14,2024-02-15,Dakar,AFRIQUE - EVASION & CANAL+,AFRIQUE - EVASION & CANAL+,Réseau en propre,Boutiques CANAL+,Cash,0,2,3,3.666667,4
982349,51720983201,2021-09-11,2021-10-01,PAS_ODD,1.0,2021-10-01,2021-09-30,2022-09-30,Dakar,AFRIQUE - ACCESS,AFRIQUE - ACCESS,Réseau en propre,Boutiques CANAL+,Prélèvement RIB,1,-19,3,-19.000000,4
982350,51720983201,2022-09-11,2022-10-01,PAS_ODD,1.0,2022-10-01,2022-09-30,2023-09-30,Dakar,AFRIQUE - ACCESS,AFRIQUE - ACCESS,Réseau en propre,Boutiques CANAL+,Prélèvement RIB,1,-19,3,-19.000000,4


In [10]:
df['Cluster_8'].unique()

array([4, 0, 3, 1, 7, 2, 5, 6])

In [6]:
df2 = file_to_dataframe(antoine + "df_Données_Reabos_odd_final.csv")